In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import torch
from torch import *

#let me be on my mac
def cuda(self, device=None, non_blocking=False) : return self
torch.Tensor.cuda = cuda

In [ ]:
from lib.callbacks import *
from lib.model import *
from lib.data import *
from lib.optimizers import *
from pathlib import Path

## create basemodel on imagenette

In [ ]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)
#path = untar_data(datasets.URLs.IMAGENETTE_160)
path

In [ ]:
size = 128
bs   = 64

tfms     = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), PilRandomFlip(), to_byte_tensor, to_float_tensor]
val_tfms = [make_rgb, CenterCrop(size), to_byte_tensor, to_float_tensor]
files    = ImageList.from_files(path, tfms=tfms)

sd       = SplitData.split_by_func(files, partial(grandparent_splitter, valid_name='val'))
data     = label_train_valid_data(sd, parent_labeler, proc_y=CategoryProcessor())
data.valid.x.tfms  = val_tfms

imagenette_features = max(data.train.y)+1
print(f"number of training, validation images: {len(data.train)},  {len(data.valid)}")
print(f"imagenette_features:{imagenette_features}")

train_dl,valid_dl = ( DataLoader(data.train, batch_size=bs,   num_workers=4, shuffle=True),
                      DataLoader(data.valid, batch_size=bs*2, num_workers=4))
databunch = DataBunch(train_dl, valid_dl, c_in=3, c_out=imagenette_features)

In [ ]:
layers_sizes = [64,64,128,256]
layer = partial( conv_layer, stride=2, bn=True, zero_bn=False, act=partial(torch.nn.ReLU,inplace=True) )
model = get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer)
init_cnn( model )

cbfs_base = [TrainableModelCallback, TrainEvalCallback, OptimizerCallback, 
#        partial(ParamScheduler, 'lr', sched),
        partial(BatchTransformXCallback, norm_imagenette),
#        partial(MixUp,α=0.4),
        
        #CudaCallback,
        ProgressCallback,
       ]
cbfs = cbfs_base.copy() + [Recorder, partial(AvgStatsCallback,[accuracy])]
cbfs_lr_Finder = cbfs_base.copy() + [LR_Finder]

sched = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)]) 

In [ ]:
model_grads_summary(model)
#xb,_ = getFirstbatch( model, databunch, partial(BatchTransformXCallback, tfm = norm_imagenette))
#model_summary(model, xb, only_leaves=True, print_mod=False)
print(f"\nmodel hierarchy:\n{model}")

In [ ]:
learn = Learner( model, databunch, loss_func=LabelSmoothingCrossEntropy())
%time learn.fit(5, opt=Adam(sched,max_lr=3e-4, moms=(0.85,0.95), max_wd = 1e-4), cb_funcs=cbfs)

In [ ]:
save_model(path, learn.model)

## Trains Pets dataset 

In [ ]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [ ]:
list(pets.iterdir())[:10]

In [ ]:
pets_path = pets/'images'
list(pets_path.iterdir())[:10]

In [ ]:
import re
def random_splitter(fn, p_valid): return random.random() < p_valid
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

files = ImageList.from_files(pets_path, tfms=tfms)
sd    = SplitData.split_by_func(files, partial(random_splitter, p_valid=0.1))

proc  = CategoryProcessor()
data  = label_train_valid_data(sd, pet_labeler, proc_y=proc)
data.valid.x.tfms = val_tfms

pets_features     = len(proc.vocab)
print(f"number of training, validation images: {len(data.train)},  {len(data.valid)}")
print(f"pets_features:{pets_features}")

train_dl,valid_dl = ( DataLoader(data.train, batch_size=bs,   num_workers=4, shuffle=True),
                      DataLoader(data.valid, batch_size=bs*2, num_workers=4))
databunch = DataBunch(train_dl, valid_dl, c_in=3, c_out=pets_features)

In [ ]:
print(f"categories:\n{ ', '.join(proc.vocab) }")

In [ ]:
#train with from scratch
model = get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer)
init_cnn( model )

In [ ]:
#xb,_ = getFirstbatch( learn.model, databunch, partial(BatchTransformXCallback, tfm = norm_imagenette))
#model_summary(model, xb, only_leaves=True, print_mod=False)
print(f"model hierarchy:\n{model}")
model_grads_summary(model)

In [ ]:
learn = Learner( model, databunch, loss_func=LabelSmoothingCrossEntropy() )
cbfs  = cbfs_base.copy() + [Recorder, partial(AvgStatsCallback,[accuracy])]
%time learn.fit(2, opt=Adam(sched,max_lr=3e-4, moms=(0.85,0.95), max_wd = 1e-4), cb_funcs=cbfs)

## use pretrained imagewoff model for training with gradual unfreezing

In [ ]:
print(f"path to pretrained model:{path}")

In [ ]:
#load pretrained on imagewoof
model = get_cnn_model(layers_sizes, 3, imagenette_features, layer)
load_model(path, model)

print(f"model hierarchy:\n{model}")
model_grads_summary(model)

In [ ]:
model = adapt_model(model, databunch, norm=norm_imagenette)

In [ ]:
freeze(model)
model_grads_summary(model)

In [ ]:
cbfs  = cbfs_base.copy() + [Recorder, partial(AvgStatsCallback,[accuracy])]
learn = Learner( model, databunch, loss_func=LabelSmoothingCrossEntropy() )

In [ ]:
%time learn.fit(1, opt=Adam(sched,max_lr=1e-2, moms=(0.85,0.95), max_wd = 1e-4), cb_funcs=cbfs)

In [ ]:
unfreeze(learn.model)
model_grads_summary(model)

In [ ]:
learn.fit(1, opt=Adam(sched,max_lr=5e-5, moms=(0.85,0.95), max_wd = 1e-6), cb_funcs=cbfs)